In [2]:
import tensorflow as tf

import pandas as pd
import numpy as np #데이터 배열화
import os #경로 설정용 
import keras #딥러닝용 패키지
import random #데이터 분산할 때 쓸 랜덤
import cv2 #이미지 읽기용
import math #연산용
import seaborn as sns #matplotlib에 다양한 시각화 기능이 추가된 패키지
from sklearn.metrics import confusion_matrix #분류의 정확성 평가
from sklearn.preprocessing import LabelBinarizer #데이터 전처리용
from sklearn.model_selection import train_test_split #데이터 분할용

import matplotlib.pyplot as plt #데이터 시각화용

from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Convolution2D,BatchNormalization #사용할 BN, ConV2 등의 계층
from tensorflow.keras.layers import Flatten,MaxPooling2D,Dropout #사용할 레이어

from tensorflow.keras.applications import VGG16 
from tensorflow.keras.applications.densenet import preprocess_input #tensor나 numpy배열 전처리용

from tensorflow.keras.preprocessing import image #이미지 데이터를 실시간으로 처리하기 위한 도구
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array #이미지 편집을 위한 제너레이터(Affine Transform)

from tensorflow.keras.models import Model #교육 및 추론 기능이 있는 개체로 레이어를 그룹화
from keras.models import Sequential

from tensorflow.keras.optimizers import Adam #Adam 옵티마이저 사용. loss는 categorical_crossentropy 사용

#체크포인트를 두고 저장 + metric이 중지되면 학습률을 감소
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau 

import warnings #경고 무시
warnings.filterwarnings("ignore")

In [3]:
image_size=32

In [4]:
data=[]
labels=[]
random.seed(42)
imagePaths = sorted(list(os.listdir("./input/emotion-project-2/train/")))
random.shuffle(imagePaths)
print(imagePaths)

for img in imagePaths:
    path=sorted(list(os.listdir("./input/emotion-project-2/train/"+img)))
    for i in path:
        image = cv2.imread("./input/emotion-project-2/train/"+img+'/'+i)
        image = cv2.resize(image, (image_size,image_size))
        image = img_to_array(image)
        data.append(image)
        l = label = img
        labels.append(l)

['neutral', 'fear', 'happy', 'sad', 'angry', 'surprise']


In [5]:
data = np.array(data, dtype="float64") / 255.0
labels = np.array(labels)
mlb = LabelBinarizer()
labels = mlb.fit_transform(labels)
print(labels[0])

[0 0 0 1 0 0]


In [6]:
(xtrain,xtest,ytrain,ytest)=train_test_split(data,labels,test_size=0.2,random_state=42)
print(xtrain.shape, xtest.shape)

(50890, 32, 32, 3) (12723, 32, 32, 3)


In [7]:
datagen = ImageDataGenerator(zoom_range = 0.2, horizontal_flip=True, shear_range=0.2) #이미지 가공(학습률 향상을 위한 augmentation)
datagen.fit(xtrain)

In [8]:
#dense relu문 추가, dropout 0.2로 변경

model = VGG16(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))
x=Flatten()(model.output)
x=Dense(1024)(x)
x=Dropout(0.2)(x)
x=Dense(512,activation='relu')(x)
x=Dropout(0.2)(x)
output=Dense(6,activation='softmax')(x) # because we have to predict the AUC
model=Model(model.input,output)

model.compile(
    optimizer = Adam(lr=0.0001), 
    loss='categorical_crossentropy', 
    metrics=['accuracy']
  )

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0     

In [9]:
history = model.fit(datagen.flow(xtrain, ytrain, batch_size=32),
               epochs=20,
               validation_data=(xtest, ytest))

Epoch 1/20
1591/1591 [==============================] - 47s 28ms/step - loss: 1.5881 - accuracy: 0.3385 - val_loss: 1.2589 - val_accuracy: 0.5117
Epoch 2/20
1591/1591 [==============================] - 42s 27ms/step - loss: 1.2153 - accuracy: 0.5270 - val_loss: 1.0868 - val_accuracy: 0.5909
Epoch 3/20
1591/1591 [==============================] - 42s 27ms/step - loss: 1.0876 - accuracy: 0.5889 - val_loss: 1.0715 - val_accuracy: 0.5986
Epoch 4/20
1591/1591 [==============================] - 42s 27ms/step - loss: 1.0006 - accuracy: 0.6231 - val_loss: 0.9656 - val_accuracy: 0.6329
Epoch 5/20
1591/1591 [==============================] - 43s 27ms/step - loss: 0.9220 - accuracy: 0.6550 - val_loss: 0.8997 - val_accuracy: 0.6631
Epoch 6/20
1591/1591 [==============================] - 43s 27ms/step - loss: 0.8465 - accuracy: 0.6876 - val_loss: 0.8906 - val_accuracy: 0.6740
Epoch 7/20
1591/1591 [==============================] - 43s 27ms/step - loss: 0.7649 - accuracy: 0.7224 - val_loss: 0.8779 -

In [10]:
model.evaluate(xtest, ytest, batch_size = 32)

398/398 [==============================] - 3s 7ms/step - loss: 0.6203 - accuracy: 0.8243


[0.6202706098556519, 0.824333906173706]

In [ ]:
model.evaluate(xtrain, ytrain, batch_size = 32)

 578/1591 [=========>....................] - ETA: 6s - loss: 0.1646 - accuracy: 0.9459

In [ ]:
model.save('0425_adam_f64.h5')